# **PART-1: Dense Network**



In [ ]:
%load_ext tensorboard
import tensorflow as tf
import datetime

# Clear any logs from previous runs
!rm -rf ./logs/


In [ ]:
%reload_ext tensorboard


In [ ]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from torchvision import datasets, transforms
import numpy as np
from google.colab import drive
from torch.utils.tensorboard import SummaryWriter  # Corrected import statement




# Load the data from the file
data = np.load('emnist_letters.npz')

# Access the arrays containing images and labels
train_images = data['train_images']
train_labels = data['train_labels']
val_images = data['validate_images']
val_labels = data['validate_labels']
test_images = data['test_images']
test_labels = data['test_labels']


# Create datasets
train_dataset = TensorDataset(torch.tensor(train_images), torch.tensor(train_labels))
val_dataset = TensorDataset(torch.tensor(val_images), torch.tensor(val_labels))
test_dataset = TensorDataset(torch.tensor(test_images), torch.tensor(test_labels))

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=64)
test_loader = DataLoader(test_dataset, batch_size=64)



device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")


In [ ]:


# Model Definition
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        # self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(28 * 28, 300, dtype=torch.float32)  # One neuron per feature- each input pixel is a feature, 28 * 28  pixel images, therefore 28 * 28  neurons
        self.fc2 = nn.Linear(300,150, dtype=torch.float32)  # arbitrarily selecting nerons for each layer, should adjust during model tuning-
        self.fc3 = nn.Linear(150, 50, dtype=torch.float32)
        self.fc4 = nn.Linear(50, 27, dtype=torch.float32)  # 27 Classes present in dataset

    def forward(self, x):
        # x = self.flatten(x)
        # Cast input data to torch.float32
        x = x.to(torch.float32)
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = torch.relu(self.fc3(x))
        x = self.fc4(x)
        return x


model = NeuralNetwork()
print(model)

# Inspect a sample of the data
for data, target in train_loader:
    print("Target shape:", target.shape)
    print("Target dataset:", target)
    break

In [ ]:

# Training
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

# Create a directory for TensorBoard logs
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
os.makedirs(log_dir, exist_ok=True)

# Create a SummaryWriter for TensorBoard
writer = SummaryWriter(log_dir=log_dir)

def train(model, train_loader, optimizer, criterion, epochs):
    for epoch in range(epochs):
        model.train()
        epoch_loss = 0.0  # Track epoch loss
        for batch_idx, (data, target) in enumerate(train_loader):
            optimizer.zero_grad()
            output = model(data)

            # Convert one-hot encoded target to class labels (1D tensor)
            target_labels = torch.nonzero(target, as_tuple=True)[1]

            # Calculate loss using class labels
            loss = criterion(output, target_labels)

            loss.backward()
            optimizer.step()

            epoch_loss += loss.item()  # Accumulate batch loss

        # Print epoch results
        print('Epoch {} - Loss: {:.6f}'.format(epoch, epoch_loss / len(train_loader)))
        # Log the training loss to TensorBoard
        writer.add_scalar('Loss/train', epoch_loss / len(train_loader), epoch)



In [ ]:
def test(model, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            output = model(data)
            # Convert one-hot encoded target to class labels (1D tensor)
            target_labels = torch.nonzero(target, as_tuple=True)[1]
            test_loss += criterion(output, target_labels).item()
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target_labels.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))


In [ ]:
train(model, train_loader, optimizer, criterion, epochs=10)
# Close the SummaryWriter
writer.close()
test(model, test_loader)

# Graphing the data from the Dense network

First, lets generate confusion matrix of the dense network's classifications

We'll start by calculating the number of true and false positives

In [ ]:
import numpy as np

# Generate predictions for the test set
def generate_predictions(model, test_loader):
    model.eval()
    all_predictions = []
    all_targets = []
    with torch.no_grad():
        for data, target in test_loader:
            output = model(data)
            all_predictions.extend(output.argmax(dim=1).cpu().numpy())
            all_targets.extend(target.cpu().numpy())
    return np.array(all_predictions), np.array(all_targets)
def compute_tp_fp(predictions, targets, class_label):
    # Convert one-hot encoded targets to class labels
    target_labels = np.argmax(targets, axis=1)

    # Compute True Positives (TP) and False Positives (FP) for the specified class label
    tp = np.sum((predictions == class_label) & (target_labels == class_label))
    fp = np.sum((predictions == class_label) & (target_labels != class_label))

    return tp, fp

# Generate predictions for the test set
test_predictions, test_targets = generate_predictions(model, test_loader)

# Compute TP and FP for each class
for class_label in range(27):
    tp, fp = compute_tp_fp(test_predictions, test_targets, class_label)
    print(f"Class {class_label}: TP={tp}, FP={fp}")

#Confusion Matrix

In [ ]:

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.metrics import confusion_matrix

# Generate predictions for the test set
test_predictions, test_targets = generate_predictions(model, test_loader)

# Convert one-hot encoded targets to class labels
test_targets_single = np.argmax(test_targets, axis=1)

# Compute confusion matrix
cm = confusion_matrix(test_targets_single, test_predictions)

# Generate labels for the letters using Unicode
letter_labels = [chr(ord('A') + i) for i in range(26)]

# Plot confusion matrix as heatmap with x-axis on top and "plasma" colormap
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt="d", cmap="plasma", xticklabels=letter_labels, yticklabels=letter_labels)
plt.xlabel("Predicted Letter")
plt.ylabel("True Letter")
plt.title("Confusion Matrix")
plt.show()


In [ ]:
%tensorboard --logdir logs/fit

#Performance Comparison between Dense Network and OPIUM based Classifier

The Dense Network used 3 hidden layers but the OPIUM based classifier used 10,000 hidden layers. Even with the huge increase in the hidden layers the accuracy of the OPIUM based classifier on the letters dataset remained at (85.15% ± 0.12%), while the dense network had a much better accuracy of 91%. In comparison with the OPIUM based classifier the dense network is more compact and runs more efficiently.

# **PART-2: Convolutional Network**

In [ ]:
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard
import datetime
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, Activation, BatchNormalization, MaxPooling2D, Dropout,Flatten, Dense
from tensorflow.keras.models import load_model

In [ ]:


# Clear any logs from previous runs
!rm -rf ./logs/
# Load the data from the file
data = np.load('emnist_letters.npz')

# Access the arrays containing images and labels
train_images = data['train_images']
train_labels = data['train_labels']
validate_images = data['validate_images']
validate_labels = data['validate_labels']
test_images = data['test_images']
test_labels = data['test_labels']


In [ ]:
# Define the log directory for TensorBoard
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

# Load the TensorBoard notebook extension
%load_ext tensorboard

# Define TensorBoard callback
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

In [ ]:
num_train_images = train_images.shape[0]
print("Number of images in the training dataset:", num_train_images)

In [ ]:
# Count occurrences of each label
label_counts = np.sum(train_labels, axis=0)

# Plot the distribution of labels
plt.bar(range(len(label_counts)), label_counts)
plt.xlabel('Label')
plt.ylabel('Count')
plt.title('Distribution of Labels in Training Dataset')
plt.show()

In [ ]:
print(train_images.shape)
print(validate_images.shape)

In [ ]:
!pip install tensorflow

In [ ]:
# Define the strategy
strategy = tf.distribute.MirroredStrategy()
with strategy.scope():
    model = Sequential()

    # Reshape the input images to their original 28x28 shape (assuming original shape was 28x28)
    model.add(tf.keras.layers.Reshape((28, 28, 1), input_shape=(784,)))

    # Feature Learning Layers
    model.add(Conv2D(32,                  # Number of filters/Kernels
                     (3,3),               # Size of kernels (3x3 matrix)
                     strides = 1,         # Step size for sliding the kernel across the input (1 pixel at a time).
                     padding = 'same'    # 'Same' ensures that the output feature map has the same dimensions as the input by padding zeros around the input.
                    ))
    model.add(Activation('relu'))# Activation function
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size = (2,2), padding = 'same'))
    model.add(Dropout(0.2))

    model.add(Conv2D(64, (5,5), padding = 'same'))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size = (2,2), padding = 'same'))
    model.add(Dropout(0.2))

    model.add(Conv2D(128, (3,3), padding = 'same'))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size = (2,2), padding = 'same'))
    model.add(Dropout(0.3))

    # Flattening tensors
    model.add(Flatten())

    # Fully-Connected Layers
    model.add(Dense(2048))
    model.add(Activation('relu'))
    model.add(Dropout(0.5))

    # Output Layer
    model.add(Dense(27, activation = 'softmax')) # Classification layer

In [ ]:
model.compile(optimizer = tf.keras.optimizers.RMSprop(0.0001), # 1e-4
              loss = 'categorical_crossentropy', # Ideal for multiclass tasks
              metrics = ['accuracy']) # Evaluation metric

# Defining an Early Stopping and Model Checkpoints
early_stopping = EarlyStopping(monitor = 'val_accuracy',
                              patience = 5, mode = 'max',
                              restore_best_weights = True)

checkpoint = ModelCheckpoint('best_model.h5',
                            monitor = 'val_accuracy',
                            save_best_only = True)

In [ ]:
# Define the number of epochs
num_epochs = 50

# Fit the model to the training data
history = model.fit(train_images, train_labels,
                    epochs=num_epochs,
                    validation_data=(validate_images, validate_labels),
                    callbacks=[early_stopping, checkpoint, tensorboard_callback])


In [ ]:
%tensorboard --logdir logs/fit

In [ ]:
# Load the best model
best_model = load_model('best_model.h5')

# Evaluate the best model on test data
test_loss, test_accuracy = best_model.evaluate(test_images, test_labels)

print('Test Loss:', test_loss)
print('Test Accuracy:', test_accuracy)

Generate True and False Positives for Convolutional Network

In [ ]:
import numpy as np
from tensorflow.keras.models import load_model

# Load the best model
best_model = load_model('best_model.h5')

# Generate predictions for the test set
def generate_predictions(model, test_images):
    predictions = model.predict(test_images)
    return np.argmax(predictions, axis=1)

# Load the test data
test_images = data['test_images']
test_labels = data['test_labels']

# Flatten the test labels
test_labels_flat = np.argmax(test_labels, axis=1)

# Compute TP and FP for each class
def compute_tp_fp(predictions, targets, class_label):
    # Compute True Positives (TP) and False Positives (FP) for the specified class label
    tp = np.sum((predictions == class_label) & (targets == class_label))
    fp = np.sum((predictions == class_label) & (targets != class_label))
    return tp, fp

# Generate predictions for the test set
test_predictions = generate_predictions(best_model, test_images)

# Compute TP and FP for each class
for class_label in range(27):
    tp, fp = compute_tp_fp(test_predictions, test_labels_flat, class_label)
    print(f"Class {class_label}: TP={tp}, FP={fp}")


#Confusion Matrix

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Define class labels (assuming class labels are represented as integers from 0 to 25, corresponding to letters A to Z)
class_labels = range(26)
letter_labels = [chr(ord('A') + i) for i in class_labels]

# Compute confusion matrix
cm = confusion_matrix(test_labels_flat, test_predictions)

# Plot confusion matrix as heatmap with "viridis" colormap
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt="d", cmap="viridis", xticklabels=letter_labels, yticklabels=letter_labels)
plt.xlabel("Predicted Letter")
plt.ylabel("True Letter")
plt.title("Confusion Matrix")
plt.show()

#Performance Comparison between Dense Network and CNN

The dense network performed at 91% accuracy while the convolution network performed a bit better at 94%. The models both struggled in the same areas and both had most of their misidentifications in the same place. Similar letters were frequently misidentified as each other- for example both of the networks had the most issues misidentifying the letter I as the letter L, and vice versa. The second highest misidentifications were Q and G. Overall though, the Convolution network was more consistent in identifying letters, with far more pairs of letters at 0 total misidentifications.


# **Part-3: GAN**

In [ ]:
import numpy as np
from torch.utils.data import Dataset, DataLoader
import argparse
import os
import random
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torchvision.utils as vutils
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML
from torch.utils.tensorboard import SummaryWriter
writer = SummaryWriter()

# Load the data from the file

data = np.load('emnist_letters.npz')

# Access the arrays containing images and labels
train_images = data['train_images']
train_labels = data['train_labels']
validate_images = data['validate_images']
validate_labels = data['validate_labels']
test_images = data['test_images']
test_labels = data['test_labels']


# Concatenate images and labels arrays
all_images = np.concatenate([train_images, validate_images, test_images], axis=0)
all_labels = np.concatenate([train_labels, validate_labels, test_labels], axis=0)


In [ ]:
# Number of workers for dataloader
workers = 2

# Batch size during training
batch_size = 128

# Spatial size of training images. All images will be resized to this
#   size using a transformer.
image_size = 64

# Number of channels in the training images. For color images this is 3
nc = 1

# Size of z latent vector (i.e. size of generator input)
nz = 100

# Size of feature maps in generator
ngf = 28

# Size of feature maps in discriminator
ndf = 28

# Number of training epochs
num_epochs = 50

# Learning rate for optimizers
lr = 0.0002

# Beta1 hyperparameter for Adam optimizers
beta1 = 0.5

# Number of GPUs available. Use 0 for CPU mode.
ngpu = 1

In [ ]:


class CustomDataset(Dataset):
    def __init__(self, images, labels, transform=None):
        self.images = images
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        image = self.images[idx].reshape(28, 28)  # Reshape flattened image to 2D
        label = self.labels[idx]

        if self.transform:
            image = self.transform(image)

        return image, label

# Transform for image preprocessing
transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize(image_size),
    transforms.CenterCrop(image_size),
    transforms.ToTensor()
])

# Create custom dataset instances
train_dataset = CustomDataset(all_images, all_labels, transform=transform)


# Create dataloaders
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=workers)

In [ ]:
# Decide which device we want to run on
device = torch.device("cuda:0" if (torch.cuda.is_available() and ngpu > 0) else "cpu")
real_batch = next(iter(train_dataloader))
plt.figure(figsize=(8,8))
plt.axis("off")
plt.title("Training Images")
plt.imshow(np.transpose(vutils.make_grid(real_batch[0].to(device)[:64], padding=2, normalize=True).cpu(),(1,2,0)))
plt.show()

In [ ]:
# custom weights initialization called on ``netG`` and ``netD``
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        nn.init.normal_(m.weight.data, 0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        nn.init.normal_(m.weight.data, 1.0, 0.02)
        nn.init.constant_(m.bias.data, 0)

In [ ]:
# Generator Code

class Generator(nn.Module):
    def __init__(self, ngpu):
        super(Generator, self).__init__()
        self.ngpu = ngpu
        self.main = nn.Sequential(
            # input is Z, going into a convolution
            nn.ConvTranspose2d( nz, ngf * 8, 4, 1, 0, bias=False),
            nn.BatchNorm2d(ngf * 8),
            nn.ReLU(True),
            # state size. ``(ngf*8) x 4 x 4``
            nn.ConvTranspose2d(ngf * 8, ngf * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf * 4),
            nn.ReLU(True),
            # state size. ``(ngf*4) x 8 x 8``
            nn.ConvTranspose2d( ngf * 4, ngf * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf * 2),
            nn.ReLU(True),
            # state size. ``(ngf*2) x 16 x 16``
            nn.ConvTranspose2d( ngf * 2, ngf, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf),
            nn.ReLU(True),
            # state size. ``(ngf) x 32 x 32``
            nn.ConvTranspose2d( ngf, nc, 4, 2, 1, bias=False),
            nn.Tanh()
            # state size. ``(nc) x 64 x 64``
        )

    def forward(self, input):
        return self.main(input)

In [ ]:
# Create the generator
netG = Generator(ngpu).to(device)

# Handle multi-GPU if desired
if (device.type == 'cuda') and (ngpu > 1):
    netG = nn.DataParallel(netG, list(range(ngpu)))

# Apply the ``weights_init`` function to randomly initialize all weights
#  to ``mean=0``, ``stdev=0.02``.
netG.apply(weights_init)

# Print the model
print(netG)

In [ ]:
class Discriminator(nn.Module):
    def __init__(self, ngpu):
        super(Discriminator, self).__init__()
        self.ngpu = ngpu
        self.main = nn.Sequential(
            # input is ``(nc) x 64 x 64``
            nn.Conv2d(nc, ndf, 4, 2, 1, bias=False),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. ``(ndf) x 32 x 32``
            nn.Conv2d(ndf, ndf * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 2),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. ``(ndf*2) x 16 x 16``
            nn.Conv2d(ndf * 2, ndf * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 4),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. ``(ndf*4) x 8 x 8``
            nn.Conv2d(ndf * 4, ndf * 8, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 8),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. ``(ndf*8) x 4 x 4``
            nn.Conv2d(ndf * 8, 1, 4, 1, 0, bias=False),
            nn.Sigmoid()
        )

    def forward(self, input):
        return self.main(input)

In [ ]:
# Create the Discriminator
netD = Discriminator(ngpu).to(device)

# Handle multi-GPU if desired
if (device.type == 'cuda') and (ngpu > 1):
    netD = nn.DataParallel(netD, list(range(ngpu)))

# Apply the ``weights_init`` function to randomly initialize all weights
# like this: ``to mean=0, stdev=0.2``.
netD.apply(weights_init)

# Print the model
print(netD)

In [ ]:
# Initialize the ``BCELoss`` function
criterion = nn.BCELoss()

# Create batch of latent vectors that we will use to visualize
#  the progression of the generator
fixed_noise = torch.randn(64, nz, 1, 1, device=device)

# Establish convention for real and fake labels during training
real_label = 1.
fake_label = 0.

# Setup Adam optimizers for both G and D
optimizerD = optim.Adam(netD.parameters(), lr=lr, betas=(beta1, 0.999))
optimizerG = optim.Adam(netG.parameters(), lr=lr, betas=(beta1, 0.999))

In [ ]:
# Training Loop

# Lists to keep track of progress
img_list = []
G_losses = []
D_losses = []
iters = 0

%load_ext tensorboard


print("Starting Training Loop...")
# For each epoch
for epoch in range(num_epochs):
    # For each batch in the dataloader
    for i, data in enumerate(train_dataloader, 0):

        ############################
        # (1) Update D network: maximize log(D(x)) + log(1 - D(G(z)))
        ###########################
        ## Train with all-real batch
        netD.zero_grad()
        # Format batch
        real_cpu = data[0].to(device)
        b_size = real_cpu.size(0)
        label = torch.full((b_size,), real_label, dtype=torch.float, device=device)
        # Forward pass real batch through D
        output = netD(real_cpu).view(-1)
        # Calculate loss on all-real batch
        errD_real = criterion(output, label)
        # Calculate gradients for D in backward pass
        errD_real.backward()
        D_x = output.mean().item()

        ## Train with all-fake batch
        # Generate batch of latent vectors
        noise = torch.randn(b_size, nz, 1, 1, device=device)
        # Generate fake image batch with G
        fake = netG(noise)
        label.fill_(fake_label)
        # Classify all fake batch with D
        output = netD(fake.detach()).view(-1)
        # Calculate D's loss on the all-fake batch
        errD_fake = criterion(output, label)
        # Calculate the gradients for this batch, accumulated (summed) with previous gradients
        errD_fake.backward()
        D_G_z1 = output.mean().item()
        # Compute error of D as sum over the fake and the real batches
        errD = errD_real + errD_fake
        # Update D
        optimizerD.step()

        ############################
        # (2) Update G network: maximize log(D(G(z)))
        ###########################
        netG.zero_grad()
        label.fill_(real_label)  # fake labels are real for generator cost
        # Since we just updated D, perform another forward pass of all-fake batch through D
        output = netD(fake).view(-1)
        # Calculate G's loss based on this output
        errG = criterion(output, label)
        # Calculate gradients for G
        errG.backward()
        D_G_z2 = output.mean().item()
        # Update G
        optimizerG.step()

        # Output training stats
        if i % 50 == 0:
            print('[%d/%d][%d/%d]\tLoss_D: %.4f\tLoss_G: %.4f\tD(x): %.4f\tD(G(z)): %.4f / %.4f'
                  % (epoch, num_epochs, i, len(train_dataloader),
                     errD.item(), errG.item(), D_x, D_G_z1, D_G_z2))

        # Save Losses for plotting later
        G_losses.append(errG.item())
        D_losses.append(errD.item())

        # Check how the generator is doing by saving G's output on fixed_noise
        if (iters % 500 == 0) or ((epoch == num_epochs-1) and (i == len(train_dataloader)-1)):
            with torch.no_grad():
                fake = netG(fixed_noise).detach().cpu()
            img_list.append(vutils.make_grid(fake, padding=2, normalize=True))


        # Log scalar values
        writer.add_scalar('Loss/Discriminator', errD.item(), global_step=iters)
        writer.add_scalar('Loss/Generator', errG.item(), global_step=iters)
        writer.add_scalar('Performance/D(x)', D_x, global_step=iters)
        writer.add_scalar('Performance/D(G(z1))', D_G_z1, global_step=iters)
        writer.add_scalar('Performance/D(G(z2))', D_G_z2, global_step=iters)

        # Log images generated by the GAN
        if iters % 500 == 0 or ((epoch == num_epochs-1) and (i == len(train_dataloader)-1)):
            with torch.no_grad():
                fake = netG(fixed_noise).detach().cpu()
            img_grid = vutils.make_grid(fake, padding=2, normalize=True)
            writer.add_image('Generated Images', img_grid, global_step=iters)

        iters += 1

In [ ]:
%tensorboard --logdir runs

In [ ]:
plt.figure(figsize=(10,5))
plt.title("Generator and Discriminator Loss During Training")
plt.plot(G_losses,label="G")
plt.plot(D_losses,label="D")
plt.xlabel("iterations")
plt.ylabel("Loss")
plt.legend()
plt.show()

In [ ]:
fig = plt.figure(figsize=(8,8))
plt.axis("off")
ims = [[plt.imshow(np.transpose(i,(1,2,0)), animated=True)] for i in img_list]
ani = animation.ArtistAnimation(fig, ims, interval=1000, repeat_delay=1000, blit=True)

HTML(ani.to_jshtml())

In [ ]:
# Grab a batch of real images from the dataloader
real_batch = next(iter(train_dataloader))

# Plot the real images
plt.figure(figsize=(15,15))
plt.subplot(1,2,1)
plt.axis("off")
plt.title("Real Images")
plt.imshow(np.transpose(vutils.make_grid(real_batch[0].to(device)[:64], padding=5, normalize=True).cpu(),(1,2,0)))

# Plot the fake images from the last epoch
plt.subplot(1,2,2)
plt.axis("off")
plt.title("Fake Images")
plt.imshow(np.transpose(img_list[-1],(1,2,0)))
plt.show()

From the above output we can say that the fake images generated by the GAN are almost indistinguishable for some of the letters like "Y",'a',"E","S","k", etc.

#Contributions

Shayan Darian: 20%

Sai Chaitanya Kilambi: 20%

Bharti Moryani: 20%

Yashvi Navadia: 20%

Drew Williams: 20%
